In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.amp import autocast, GradScaler
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
import gc
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import random
from torch.optim.lr_scheduler import CosineAnnealingLR
import matplotlib.pyplot as plt

# Set memory optimization flags and reproducibility
torch.backends.cudnn.benchmark = True
torch.cuda.empty_cache()
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
class MultimodalDataset(torch.utils.data.Dataset):
    def __init__(self, emg_data, eeg_data, labels):
        self.emg_data = torch.FloatTensor(emg_data)
        self.eeg_data = torch.FloatTensor(eeg_data)
        self.labels = torch.LongTensor(labels)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.emg_data[idx], self.eeg_data[idx], self.labels[idx]

In [3]:
# Update the EMGEncoder and EEGEncoder classes with residual connections and dropout
class EMGEncoder(nn.Module):
    def __init__(self, num_channels, hidden_dim):
        super(EMGEncoder, self).__init__()
        self.input_conv = nn.Sequential(
            nn.Conv1d(num_channels, hidden_dim, kernel_size=5, padding=2),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # Residual blocks
        self.res_blocks = nn.ModuleList([
            ResidualBlock(hidden_dim, hidden_dim * 2),
            ResidualBlock(hidden_dim * 2, hidden_dim * 4)
        ])
        
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)  # [batch, channels, timesteps]
        x = self.input_conv(x)
        
        # Apply residual blocks
        for block in self.res_blocks:
            x = block(x)
            
        x = self.global_pool(x)
        return x.view(x.shape[0], -1)  # Flatten

class EEGEncoder(nn.Module):
    def __init__(self, num_channels, hidden_dim):
        super(EEGEncoder, self).__init__()
        self.input_conv = nn.Sequential(
            nn.Conv1d(num_channels, hidden_dim, kernel_size=5, padding=2),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # Residual blocks
        self.res_blocks = nn.ModuleList([
            ResidualBlock(hidden_dim, hidden_dim * 2),
            ResidualBlock(hidden_dim * 2, hidden_dim * 4)
        ])
        
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.input_conv(x)
        
        for block in self.res_blocks:
            x = block(x)
            
        x = self.global_pool(x)
        return x.view(x.shape[0], -1)

# Add ResidualBlock class
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        # Shortcut connection
        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1),
                nn.BatchNorm1d(out_channels)
            )
            
    def forward(self, x):
        residual = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += self.shortcut(residual)
        x = F.relu(x)
        return x

# Update MultimodalCNN with attention and improved fusion
class MultimodalCNN(nn.Module):
    def __init__(self, emg_channels, eeg_channels, hidden_dim, num_classes):
        super(MultimodalCNN, self).__init__()
        
        # Print initialization parameters for debugging
        print(f"Initializing MultimodalCNN with:")
        print(f"EMG channels: {emg_channels}")
        print(f"EEG channels: {eeg_channels}")
        print(f"Hidden dim: {hidden_dim}")
        print(f"Num classes: {num_classes}")
        
        self.emg_encoder = EMGEncoder(emg_channels, hidden_dim)
        self.eeg_encoder = EEGEncoder(eeg_channels, hidden_dim)
        
        feature_dim = hidden_dim * 4 * 2  # Both encoders output hidden_dim * 4
        
        # Cross-modal attention
        self.attention = CrossModalAttention(hidden_dim * 4)
        
        # Improved fusion network with skip connections
        self.fusion = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim * 4),
            nn.LayerNorm(hidden_dim * 4),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            ResidualLinear(hidden_dim * 4, hidden_dim * 2),
            nn.Dropout(0.2),
            
            ResidualLinear(hidden_dim * 2, hidden_dim),
            nn.Dropout(0.1),
            
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, emg, eeg):
        emg_features = self.emg_encoder(emg)
        eeg_features = self.eeg_encoder(eeg)
        
        # Apply cross-modal attention
        emg_attended, eeg_attended = self.attention(emg_features, eeg_features)
        
        # Concatenate attended features
        combined = torch.cat((emg_attended, eeg_attended), dim=1)
        return self.fusion(combined)

# Add CrossModalAttention class
class CrossModalAttention(nn.Module):
    def __init__(self, feature_dim):
        super(CrossModalAttention, self).__init__()
        self.query_transform = nn.Linear(feature_dim, feature_dim)
        self.key_transform = nn.Linear(feature_dim, feature_dim)
        self.value_transform = nn.Linear(feature_dim, feature_dim)
        self.scale = torch.sqrt(torch.FloatTensor([feature_dim])).to(device)
        
    def forward(self, emg_features, eeg_features):
        # EMG attending to EEG
        Q_emg = self.query_transform(emg_features)
        K_eeg = self.key_transform(eeg_features)
        V_eeg = self.value_transform(eeg_features)
        
        attention_weights = torch.matmul(Q_emg, K_eeg.transpose(-2, -1)) / self.scale
        attention_weights = F.softmax(attention_weights, dim=-1)
        emg_attended = torch.matmul(attention_weights, V_eeg)
        
        # EEG attending to EMG
        Q_eeg = self.query_transform(eeg_features)
        K_emg = self.key_transform(emg_features)
        V_emg = self.value_transform(emg_features)
        
        attention_weights = torch.matmul(Q_eeg, K_emg.transpose(-2, -1)) / self.scale
        attention_weights = F.softmax(attention_weights, dim=-1)
        eeg_attended = torch.matmul(attention_weights, V_emg)
        
        return emg_attended, eeg_attended

# Add ResidualLinear class for the fusion network
class ResidualLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(ResidualLinear, self).__init__()
        self.linear1 = nn.Linear(in_features, out_features)
        self.ln1 = nn.LayerNorm(out_features)
        self.linear2 = nn.Linear(out_features, out_features)
        self.ln2 = nn.LayerNorm(out_features)
        
        self.shortcut = nn.Sequential()
        if in_features != out_features:
            self.shortcut = nn.Linear(in_features, out_features)
            
    def forward(self, x):
        residual = x
        x = F.relu(self.ln1(self.linear1(x)))
        x = self.ln2(self.linear2(x))
        x += self.shortcut(residual)
        x = F.relu(x)
        return x

MultimodalCNN(
  (emg_encoder): EMGEncoder(
    (model): Sequential(
      (0): Conv1d(8, 128, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (9): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU()
      (11): AdaptiveAvgPool1d(output_size=1)
    )
  )
  (eeg_encoder): EEGEncoder(
    (model): Sequential(
      (0): Conv1d(6, 128, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): BatchNorm1d(128, eps

In [ ]:

# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultimodalCNN(emg_channels=8, eeg_channels=6, hidden_dim=128, num_classes=6).to(device)
print(model)

In [4]:
# Add noise to data for augmentation
def add_noise(data, noise_factor=0.05):
    noise = torch.randn(data.shape).to(data.device) * noise_factor * torch.std(data)
    return data + noise

print("Data augmentation function defined to add random noise during training")

Data augmentation function defined to add random noise during training


In [5]:
def load_and_process_data(emg_path, eeg_path, window_size=50, stride=25):
    print("Loading data...")
    
    # Load data
    emg_data = pd.read_csv(emg_path)
    eeg_data = pd.read_csv(eeg_path)
    
    # Extract features and labels
    emg_features = emg_data.iloc[:, :8].values  # Make sure we're getting exactly 8 channels
    eeg_features = eeg_data.iloc[:, :8].values  # Make sure we're getting exactly 8 channels
    
    print(f"EMG features shape: {emg_features.shape}")
    print(f"EEG features shape: {eeg_features.shape}")
    
    # Standardize the features
    print("Normalizing data...")
    emg_scaler = StandardScaler()
    eeg_scaler = StandardScaler()
    
    emg_features = emg_scaler.fit_transform(emg_features)
    eeg_features = eeg_scaler.fit_transform(eeg_features)
    
    # Create windowed data
    emg_windows = []
    eeg_windows = []
    window_labels = []
    sample_ids = []  # Track sample IDs for stratified splits
    
    # Find common samples between EMG and EEG data
    emg_samples = set(tuple(x) for x in emg_data[['subject', 'repetition', 'gesture']].drop_duplicates().values)
    eeg_samples = set(tuple(x) for x in eeg_data[['subject', 'repetition', 'gesture']].drop_duplicates().values)
    common_samples = emg_samples.intersection(eeg_samples)
    
    print(f"Found {len(common_samples)} common samples between EMG and EEG data.")
    
    for sample in common_samples:
        subject, repetition, gesture = sample
        
        # Get data for this sample
        emg_sample = emg_data[(emg_data['subject'] == subject) & 
                             (emg_data['repetition'] == repetition) & 
                             (emg_data['gesture'] == gesture)]
        
        eeg_sample = eeg_data[(eeg_data['subject'] == subject) & 
                             (eeg_data['repetition'] == repetition) & 
                             (eeg_data['gesture'] == gesture)]
        
        # Make sure both samples have data
        if len(emg_sample) == 0 or len(eeg_sample) == 0:
            continue
            
        # Extract features
        emg_sample_features = emg_sample.iloc[:, :8].values
        eeg_sample_features = eeg_sample.iloc[:, :8].values
        
        # Standardize using pre-fitted scalers
        emg_sample_features = emg_scaler.transform(emg_sample_features)
        eeg_sample_features = eeg_scaler.transform(eeg_sample_features)
        
        # Handle different lengths by using the shorter one
        min_length = min(len(emg_sample_features), len(eeg_sample_features))
        if min_length <= window_size:
            continue  # Skip if sample is too short
            
        emg_sample_features = emg_sample_features[:min_length]
        eeg_sample_features = eeg_sample_features[:min_length]
        
        # Create windows
        for i in range(0, min_length - window_size, stride):
            emg_windows.append(emg_sample_features[i:i+window_size])
            eeg_windows.append(eeg_sample_features[i:i+window_size])
            window_labels.append(gesture - 1)  # 0-indexed labels
            sample_ids.append(f"{subject}_{repetition}_{gesture}")  # Track which sample this comes from
    
    if len(emg_windows) == 0:
        raise ValueError("No valid windows could be created. Check your data alignment.")
    
    print(f"Created {len(emg_windows)} windows from {len(common_samples)} samples.")
    
    return np.array(emg_windows), np.array(eeg_windows), np.array(window_labels), np.array(sample_ids)

print("Data loading and processing function defined with windowing and standardization")

Data loading and processing function defined with windowing and standardization


In [6]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, 
                num_epochs=10, clip_grad=1.0, warmup_epochs=3, learning_rate=0.001):
    """
    Training function with improved features including warmup, early stopping, and gradient clipping
    """
   
    # Increase early stopping patience
    early_stopping_patience = 15  # Changed from 5 to 15
    best_val_loss = float('inf')
    patience_counter = 0
    min_delta = 0.001  # Minimum change in validation loss to be considered as improvement
    
    # Add learning rate warmup
    warmup_factor = 1.0 / warmup_epochs
    
    # Initialize gradient scaler for mixed precision training
    scaler = GradScaler()
    
    # Add weighted cross entropy loss to handle class imbalance
    class_counts = torch.bincount(torch.tensor([label for _, _, label in train_loader.dataset]))
    weights = 1.0 / class_counts.float()
    weights = weights / weights.sum()
    criterion = nn.CrossEntropyLoss(weight=weights.to(device), label_smoothing=0.1)


    # Add early stopping
    early_stopping_patience = 5
    best_val_loss = float('inf')
    patience_counter = 0
    
    # Keep track of metrics
    history = {
        'train_loss': [], 'train_acc': [], 'train_f1': [],
        'val_loss': [], 'val_acc': [], 'val_f1': [],
        'learning_rates': []
    }

    # Store initial learning rate
    initial_lr = learning_rate
    
    for epoch in range(num_epochs):
        # Modified learning rate warmup
        if epoch < warmup_epochs:
            lr = learning_rate * ((epoch + 1) / warmup_epochs)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
        
        # Training phase with gradient accumulation
        model.train()
        running_loss = 0.0
        all_preds = []
        all_targets = []
        
        # Add progress bar
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
        # Add gradient accumulation steps
        accumulation_steps = 2  # Accumulate gradients every 2 steps
        optimizer.zero_grad()
        
        for i, (emg_inputs, eeg_inputs, labels) in enumerate(pbar):
            # Move data to device
            emg_inputs = emg_inputs.to(device)
            eeg_inputs = eeg_inputs.to(device)
            labels = labels.to(device)
            
            # Data augmentation (add noise) in training only
            if random.random() < 0.5:  # 50% chance to apply noise
                emg_inputs = add_noise(emg_inputs, noise_factor=0.03)
                eeg_inputs = add_noise(eeg_inputs, noise_factor=0.03)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass with mixed precision
            with autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(emg_inputs, eeg_inputs)
                loss = criterion(outputs, labels)
            
            # Backward and optimize with gradient scaling
            scaler.scale(loss).backward()
            
            # Gradient clipping
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)
            
            scaler.step(optimizer)
            scaler.update()
            
            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            
            # Store predictions and labels for F1 score calculation
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())
            
            # Update progress bar
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'lr': f'{optimizer.param_groups[0]["lr"]:.6f}'
            })
            
            # Free up memory
            if i % 10 == 0:
                torch.cuda.empty_cache()
        
        # Calculate epoch metrics
        train_loss = running_loss / len(train_loader)
        train_acc = 100 * np.mean(np.array(all_preds) == np.array(all_targets))
        train_f1 = f1_score(all_targets, all_preds, average='weighted')
        
        # Validation phase
        val_loss, val_acc, val_f1 = validate_model(model, val_loader, criterion)
        
        # Store metrics
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['train_f1'].append(train_f1)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_f1'].append(val_f1)
        history['learning_rates'].append(optimizer.param_groups[0]['lr'])
          
        # Modify the early stopping check
        if val_loss < (best_val_loss - min_delta):
            best_val_loss = val_loss
            patience_counter = 0
            # Save best model
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
                'val_loss': val_loss,
                'val_acc': val_acc,
                'val_f1': val_f1,
                'history': history
            }, 'best_model_checkpoint.pth')
            print(f'Saved new best model with validation loss: {val_loss:.4f}')
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f'Early stopping triggered after {epoch+1} epochs')
                print(f'Best validation loss: {best_val_loss:.4f}')
                break
            
        # Update learning rate (except during warmup)
        if epoch >= warmup_epochs and scheduler is not None:
            scheduler.step()
        
        # Print epoch results
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Train F1: {train_f1:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, Val F1: {val_f1:.4f}')
        print(f'Learning Rate: {optimizer.param_groups[0]["lr"]:.6f}')
        print('-' * 80)

    return model, history

def validate_model(model, val_loader, criterion):
    """
    Enhanced validation function that returns loss, accuracy and F1 score
    """
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for emg_inputs, eeg_inputs, labels in val_loader:
            emg_inputs = emg_inputs.to(device)
            eeg_inputs = eeg_inputs.to(device)
            labels = labels.to(device)
            
            with autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(emg_inputs, eeg_inputs)
                loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())
    
    val_loss = val_loss / len(val_loader)
    val_acc = 100 * np.mean(np.array(all_preds) == np.array(all_targets))
    val_f1 = f1_score(all_targets, all_preds, average='weighted')
    
    return val_loss, val_acc, val_f1

Training function updated with gradient clipping and learning rate warmup.


In [7]:
# Function to evaluate on test set
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for emg_inputs, eeg_inputs, labels in test_loader:
            # Move data to device
            emg_inputs, eeg_inputs, labels = emg_inputs.to(device), eeg_inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(emg_inputs, eeg_inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            # Store predictions and labels
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())
    
    # Calculate metrics
    accuracy = 100 * np.mean(np.array(all_preds) == np.array(all_targets))
    f1 = f1_score(all_targets, all_preds, average='weighted')
    precision = precision_score(all_targets, all_preds, average='weighted')
    recall = recall_score(all_targets, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_targets, all_preds)
    
    print("Test Results:")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'confusion_matrix': conf_matrix
    }

print("Model evaluation function defined with comprehensive metrics calculation")

Model evaluation function defined with comprehensive metrics calculation


In [8]:
def visualize_model_predictions(model, test_loader, num_samples=5):
    """
    Visualize model predictions on random samples from the test set
    """
    model.eval()
    
    # Get a batch of data
    emg_samples, eeg_samples, labels = next(iter(test_loader))
    
    # Select random indices
    indices = np.random.choice(len(emg_samples), min(num_samples, len(emg_samples)), replace=False)
    
    fig, axs = plt.subplots(num_samples, 2, figsize=(15, 4*num_samples))
    
    for i, idx in enumerate(indices):
        # Get the sample
        emg_sample = emg_samples[idx].unsqueeze(0).to(device)
        eeg_sample = eeg_samples[idx].unsqueeze(0).to(device)
        true_label = labels[idx].item()
        
        # Get model prediction
        with torch.no_grad():
            output = model(emg_sample, eeg_sample)
            _, predicted = torch.max(output, 1)
            predicted_label = predicted.item()
        
        # Plot EMG data
        axs[i, 0].plot(emg_sample.cpu().numpy()[0].T)
        axs[i, 0].set_title(f'EMG Data: True={true_label}, Pred={predicted_label}')
        axs[i, 0].set_xlabel('Time Steps')
        axs[i, 0].set_ylabel('Amplitude')
        
        # Plot EEG data
        axs[i, 1].plot(eeg_sample.cpu().numpy()[0].T)
        axs[i, 1].set_title(f'EEG Data: True={true_label}, Pred={predicted_label}')
        axs[i, 1].set_xlabel('Time Steps')
        axs[i, 1].set_ylabel('Amplitude')
    
    plt.tight_layout()
    plt.savefig('sample_predictions.png')
    plt.close()
    print("Visualization saved to 'sample_predictions.png'")

def analyze_feature_importance(model, test_loader):
    """
    Analyze which channels contribute most to model predictions using feature occlusion
    """
    model.eval()
    
    # Baseline performance
    baseline_results = evaluate_model(model, test_loader)
    baseline_accuracy = baseline_results['accuracy']
    
    emg_channels = 8
    eeg_channels = 8
    
    # Results containers
    emg_importance = []
    eeg_importance = []
    
    # Test EMG channel importance
    for channel in range(emg_channels):
        # Create a modified dataloader with one channel occluded
        test_dataset_modified = []
        for emg_batch, eeg_batch, labels in test_loader:
            emg_batch_mod = emg_batch.clone()
            emg_batch_mod[:, :, channel] = 0  # Zero out the channel
            test_dataset_modified.append((emg_batch_mod, eeg_batch, labels))
        
        # Evaluate with this channel occluded
        accuracy_drop = 0
        total_batches = 0
        
        for emg_inputs, eeg_inputs, labels in test_dataset_modified:
            emg_inputs, eeg_inputs, labels = emg_inputs.to(device), eeg_inputs.to(device), labels.to(device)
            
            with torch.no_grad():
                outputs = model(emg_inputs, eeg_inputs)
                _, predicted = torch.max(outputs.data, 1)
                
                # Calculate accuracy
                correct = (predicted == labels).sum().item()
                accuracy = 100 * correct / len(labels)
                
                accuracy_drop += (baseline_accuracy - accuracy)
                total_batches += 1
        
        # Average drop across batches
        avg_drop = accuracy_drop / total_batches if total_batches > 0 else 0
        emg_importance.append(avg_drop)
    
    # Test EEG channel importance
    for channel in range(eeg_channels):
        # Create a modified dataloader with one channel occluded
        test_dataset_modified = []
        for emg_batch, eeg_batch, labels in test_loader:
            eeg_batch_mod = eeg_batch.clone()
            eeg_batch_mod[:, :, channel] = 0  # Zero out the channel
            test_dataset_modified.append((emg_batch, eeg_batch_mod, labels))
        
        # Evaluate with this channel occluded
        accuracy_drop = 0
        total_batches = 0
        
        for emg_inputs, eeg_inputs, labels in test_dataset_modified:
            emg_inputs, eeg_inputs, labels = emg_inputs.to(device), eeg_inputs.to(device), labels.to(device)
            
            with torch.no_grad():
                outputs = model(emg_inputs, eeg_inputs)
                _, predicted = torch.max(outputs.data, 1)
                
                # Calculate accuracy
                correct = (predicted == labels).sum().item()
                accuracy = 100 * correct / len(labels)
                
                accuracy_drop += (baseline_accuracy - accuracy)
                total_batches += 1
        
        # Average drop across batches
        avg_drop = accuracy_drop / total_batches if total_batches > 0 else 0
        eeg_importance.append(avg_drop)
    
    # Plot the results
    plt.figure(figsize=(12, 8))
    
    plt.subplot(1, 2, 1)
    plt.bar(range(emg_channels), emg_importance)
    plt.title('EMG Channel Importance')
    plt.xlabel('Channel')
    plt.ylabel('Accuracy Drop (%)')
    
    plt.subplot(1, 2, 2)
    plt.bar(range(eeg_channels), eeg_importance)
    plt.title('EEG Channel Importance')
    plt.xlabel('Channel')
    plt.ylabel('Accuracy Drop (%)')
    
    plt.tight_layout()
    plt.savefig('channel_importance.png')
    plt.close()
    print("Feature importance analysis saved to 'channel_importance.png'")

print("Additional visualization and analysis functions defined")

Additional visualization and analysis functions defined


In [ ]:
def main():
    try:
        # Set paths to your data files
        emg_path = 'data/processed/EMG-data.csv'
        eeg_path = 'data/processed/EEG-data.csv'
        
        # Setup initial parameters
        window_size = 50
        batch_size = 32
        learning_rate = 0.001
        num_epochs = 30
        warmup_epochs = 3
        hidden_dim = 128

        # Load and process data first
        emg_windows, eeg_windows, window_labels, sample_ids = load_and_process_data(
            emg_path, eeg_path, window_size=window_size
        )
        
        # Get the dimensions from the data
        emg_channels = emg_windows.shape[2]  # Should be 8
        eeg_channels = eeg_windows.shape[2]  # Should be 8
        num_classes = len(np.unique(window_labels))  # Should be 7
        
        print(f"EMG channels: {emg_channels}")
        print(f"EEG channels: {eeg_channels}")
        print(f"Number of classes: {num_classes}")

        # Create train/val/test split
        unique_samples = np.unique(sample_ids)
        samples_train, samples_temp = train_test_split(
            unique_samples, test_size=0.4, random_state=42
        )
        samples_val, samples_test = train_test_split(
            samples_temp, test_size=0.5, random_state=42
        )
        
        train_mask = np.isin(sample_ids, samples_train)
        val_mask = np.isin(sample_ids, samples_val)
        test_mask = np.isin(sample_ids, samples_test)
        
        # Split the data
        X_emg_train = emg_windows[train_mask]
        X_eeg_train = eeg_windows[train_mask]
        y_train = window_labels[train_mask]
        
        X_emg_val = emg_windows[val_mask]
        X_eeg_val = eeg_windows[val_mask]
        y_val = window_labels[val_mask]
        
        X_emg_test = emg_windows[test_mask]
        X_eeg_test = eeg_windows[test_mask]
        y_test = window_labels[test_mask]
        
        # Create datasets
        train_dataset = MultimodalDataset(X_emg_train, X_eeg_train, y_train)
        val_dataset = MultimodalDataset(X_emg_val, X_eeg_val, y_val)
        test_dataset = MultimodalDataset(X_emg_test, X_eeg_test, y_test)
        
        # Create data loaders
        train_loader = DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True, 
            pin_memory=True, num_workers=0
        )
        
        val_loader = DataLoader(
            val_dataset, batch_size=batch_size, shuffle=False,
            pin_memory=True, num_workers=0
        )
        
        test_loader = DataLoader(
            test_dataset, batch_size=batch_size, shuffle=False,
            pin_memory=True, num_workers=0
        )
        
        # Initialize model with correct dimensions
        model = MultimodalCNN(
            emg_channels=emg_channels,
            eeg_channels=eeg_channels,
            hidden_dim=hidden_dim,
            num_classes=num_classes
        ).to(device)
        
        # Count parameters
        total_params = sum(p.numel() for p in model.parameters())
        print(f"Model has {total_params:,} parameters")
        
        # Initialize training components
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
        scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs-warmup_epochs, eta_min=learning_rate/100)
        
        # Train the model
        trained_model, history = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=criterion,
            optimizer=optimizer,
            scheduler=scheduler,
            num_epochs=num_epochs,
            clip_grad=1.0,
            warmup_epochs=warmup_epochs,
            learning_rate=learning_rate
        )
        
        print("Training completed!")
        
        # Evaluate the model on test set
        test_results = evaluate_model(model, test_loader)
        
        print(f"Test F1 Score: {test_results['f1']:.4f}")
        print(f"Test Accuracy: {test_results['accuracy']:.2f}%")
        
        # Save final results
        final_results = {
            'test_results': test_results,
            'history': history
        }
        np.save('final_results.npy', final_results)
        
        # Plot training history
        plot_training_history(history)
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        import traceback
        traceback.print_exc()

def plot_training_history(history):
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(2, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    
    plt.subplot(2, 2, 3)
    plt.plot(history['train_f1'], label='Train F1')
    plt.plot(history['val_f1'], label='Validation F1')
    plt.title('F1 Score over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend()
    
    plt.subplot(2, 2, 4)
    plt.plot(history['learning_rates'], label='Learning Rate')
    plt.title('Learning Rate over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.yscale('log')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

if __name__ == "__main__":
    main()

Loading data...
Normalizing data...
Found 378 common samples between EMG and EEG data.
Created 19576 windows from 378 samples.
Number of classes: 7
EMG windows shape: (19576, 50, 8)
EEG windows shape: (19576, 50, 8)
Training set: 11752 samples
Validation set: 3900 samples
Test set: 3924 samples
Error occurred: name 'MultimodalNet' is not defined
Multimodal neural network implementation complete!


Traceback (most recent call last):
  File "C:\Users\work\AppData\Local\Temp\ipykernel_7220\2661198368.py", line 88, in main
    model = MultimodalNet(
            ^^^^^^^^^^^^^
NameError: name 'MultimodalNet' is not defined. Did you mean: 'MultimodalCNN'?
